# **Compliance GPT with LangChain**

## **Setup**

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [2]:
from utils.config import get_config
from utils.models import ModelName, get_model

config = get_config()

In [3]:
USER_ID = 'xmriz'
CONVERSATION_ID = 'xmriz-2021-07-01-01'

## **Define Model**

In [4]:
model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config)

## **Load Vector Store Index**

In [5]:
from databases.vector_store import RedisIndexManager

chat_store_ojk = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
chat_store_bi = RedisIndexManager(index_name='bi', embed_model=embed_model, config=config, db_id=0)

vector_store_ojk = chat_store_ojk.load_vector_index()
vector_store_bi = chat_store_bi.load_vector_index()

c:\Users\acer\miniconda3\envs\chatbot-ocbc\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## **Get Retriever**

In [13]:
from retriever.retriever_bi_ojk.retriever_bi_ojk import get_lotr_bi_ojk, get_retriever_bi_ojk

retriever = get_retriever_bi_ojk(
    vector_store=vector_store_ojk,
    embed_model=embed_model,
    llm_model=llm_model,
    config=config,
    top_k=20,
    top_n=6,
)

# retriever = get_lotr_bi_ojk(
#     vector_store_ojk=vector_store_ojk,
#     # vector_store_bi=vector_store_bi,
#     embed_model=embed_model,
#     llm_model=llm_model,
#     config=config,
#     top_k=20,
#     top_n=6,
# )

## **Create Chain**

In [14]:
from constant.ojk.prompt import CONTEXTUALIZE_Q_PROMPT_STR, QA_SYSTEM_PROMPT_STR
from databases.chat_store import RedisChatStore
from chain.rag_chain import create_chain, create_chain_with_chat_history

chat_store = RedisChatStore(k=5, config=config, db_id=1)

chain = create_chain(
    contextualize_q_prompt_str=CONTEXTUALIZE_Q_PROMPT_STR,
    qa_system_prompt_str=QA_SYSTEM_PROMPT_STR,
    retriever=retriever,
    llm_model=llm_model,
)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store,
)

## **Invoke**

#### **With Question, Answer, and Context**

In [15]:
from chain.rag_chain import get_response

response = get_response(
    chain=chain_history,
    question="Apa judul peraturan dengan nomor regulasi 2/POJK.03/2021?",
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID
)

response

{'rewrited question': 'Apa judul peraturan dengan nomor regulasi 2/POJK.03/2021?',
 'answer': 'Peraturan OJK Tentang Perubahan Kedua Atas Peraturan OJK Nomor 11-POJK.03-2020 \n\nSource: [Peraturan OJK Tentang Perubahan Kedua Atas Peraturan OJK Nomor 11-POJK.03-2020](https://www.ojk.go.id/id/regulasi/Documents/Pages/Peraturan-OJK-tentang-Perubahan-Kedua-Atas-POJK-Nomor-11-POJK.03',
 'context': '[\n  {\n    "metadata": {\n      "id": "doc:ojk:dd6b5aa5239845bcb69e8a7d60db5eb7",\n      "title": "Peraturan OJK tentang Perubahan Kedua Atas Peraturan OJK Nomor 34-POJK.03-2020",\n      "sector": "Perbankan",\n      "subsector": "BPR",\n      "regulation_type": "Peraturan OJK",\n      "regulation_number": "18/POJK.03/2021",\n      "effective_date": "10 September 2021",\n      "file_url": "https://www.ojk.go.id/id/regulasi/Documents/Pages/Peraturan-OJK-tentang-Perubahan-Kedua-Atas-Peraturan-OJK-Nomor-34-POJK.03-2020-/POJK%2018%20-%2003%20-%202021.pdf",\n      "doc_id": "380",\n      "page_number

In [16]:
print(response['context'])

[
  {
    "metadata": {
      "id": "doc:ojk:dd6b5aa5239845bcb69e8a7d60db5eb7",
      "title": "Peraturan OJK tentang Perubahan Kedua Atas Peraturan OJK Nomor 34-POJK.03-2020",
      "sector": "Perbankan",
      "subsector": "BPR",
      "regulation_type": "Peraturan OJK",
      "regulation_number": "18/POJK.03/2021",
      "effective_date": "10 September 2021",
      "file_url": "https://www.ojk.go.id/id/regulasi/Documents/Pages/Peraturan-OJK-tentang-Perubahan-Kedua-Atas-Peraturan-OJK-Nomor-34-POJK.03-2020-/POJK%2018%20-%2003%20-%202021.pdf",
      "doc_id": "380",
      "page_number": "8",
      "relevance_score": 0.9976495
    },
    "page_content": "metadata={'doc_id': 380, 'title': 'Peraturan OJK tentang Perubahan Kedua Atas Peraturan OJK Nomor 34-POJK.03-2020', 'sector': 'Perbankan', 'subsector': 'BPR', 'regulation_type': 'Peraturan OJK', 'regulation_number': '18/POJK.03/2021', 'effective_date': '10 September 2021', 'file_url': 'https://www.ojk.go.id/id/regulasi/Documents/Pages/P

#### **Streaming**

In [10]:
# from chain.chain_ojk.chain_ojk import print_answer_stream

# print_answer_stream(chain=chain, question="Siapa itu lionel messi?", user_id=USER_ID, conversation_id=CONVERSATION_ID)

Saya minta maaf, tetapi pertanyaan Anda tidak terkait dengan konteks yang diberikan. Konteks yang diberikan adalah tentang kepatuhan perbankan di Indonesia. Apakah ada pertanyaan lain yang dapat saya bantu jawab?

#### **Chat Store Monitor**

In [10]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

Human: Berapa limit transaksi dari QRIS?
AI: Limit transaksi dari QRIS adalah paling banyak Rp5.000.000,00 (lima juta rupiah) pada setiap rekening. 

Source: Surat Edaran OJK 34/SEOJK.03/2021 (https://www.ojk.go.id/id/regulasi/Documents/Pages/Buku-Panduan-Akuntansi-Perbankan-Bagi-Bank-Umum-Konvensional-/SEOJK%2034%20-%2003%20-%202021.pdf#page=736)


In [17]:
# to make the chat history empty
chat_store.clear_all()